Data preprocessing

In [ ]:
with open('endata.txt', 'r') as text_file:
    eng_sent = text_file.readlines()
eng_sent = [sent.replace('\n','') for sent in eng_sent]

In [ ]:
with open('rusdata.txt', 'r') as text_file:
  ru_sent = text_file.readlines()

In [ ]:
ru_sent = [sent.replace('\n','') for sent in ru_sent]

In [ ]:
import pandas as pd

df = pd.DataFrame({'en':eng_sent,'ru':ru_sent})

In [ ]:
training_data = df.iloc[:2643]
training_list = []

validation_data = df.iloc[2643:]
validation_list = []

In [ ]:
for i, row in training_data.iterrows():
  temp_dict = {
          'en': row.en,
          'ru': row.ru
      }
  training_list.append(temp_dict)

temp_dict_1 = {
    'translation': training_list
}

for i, row in validation_data.iterrows():
  temp_dict = {
          'en': row.en,
          'ru': row.ru
      }
  validation_list.append(temp_dict)

temp_dict_2 = {
    'translation': validation_list
}

In [ ]:
training_dataset = {}
training_dataset['train'] = temp_dict_1
training_dataset['validation'] = temp_dict_2

In [2]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"
translator = pipeline("translation", model=model_checkpoint)
translator("Это тест.")

config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/307M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.08M [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/803k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.60M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:197: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


[{'translation_text': "It's a test."}]

In [ ]:
pip install sacremoses

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-ru-en"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

In [ ]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["ru"] for ex in examples["translation"]]
    targets = [ex["en"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [ ]:
tokenized_validation_list = []
tokenized_train_list = []
tokenized_datasets = {}

validation_function_output = preprocess_function(training_dataset['validation'])
training_function_output = preprocess_function(training_dataset['train'])

In [ ]:
for i in range(len(validation_function_output['input_ids'])):
  temp_dict = {
      'input_ids' : validation_function_output['input_ids'][i],
      'attention_mask': validation_function_output['attention_mask'][i],
      'labels': validation_function_output['labels'][i]
  }
  tokenized_validation_list.append(temp_dict)

for i in range(len(training_function_output['input_ids'])):
  temp_dict = {
      'input_ids' : training_function_output['input_ids'][i],
      'attention_mask': training_function_output['attention_mask'][i],
      'labels': training_function_output['labels'][i]
  }
  tokenized_train_list.append(temp_dict)

In [ ]:
tokenized_datasets['validation'] = tokenized_validation_list
tokenized_datasets['train'] = tokenized_train_list

In [ ]:
tokenized_datasets['train'][0]

Fine tuning of the model

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

In [ ]:
batch["labels"]

In [ ]:
batch["decoder_input_ids"]

In [ ]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

In [ ]:
!pip install sacrebleu

In [ ]:
pip install evaluate

In [ ]:
import evaluate

metric = evaluate.load("sacrebleu")

In [ ]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
pip install transformers[torch]

In [ ]:
pip install accelerate -U

In [ ]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"Gopal-finetuned-custom-ru-to-en",
    evaluation_strategy="no",
    save_strategy="epoch",
    learning_rate=1e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=200,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [ ]:
 from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.evaluate(max_length=max_length)

In [ ]:
trainer.train()

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

In [ ]:
trainer.evaluate(max_length=max_length)